In [8]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

query_mapping = {
    "glucose in blood": {
        "component": "glucose",
        "system": "blood"
    },
    "bilirubin in plasma": {
        "component": "bilirubin",
        "system": "plasma"
    },
    "white blood cells count": {
        "component": "leukocytes",
        "system": "blood"
    }
}

file_path = "./loinc_dataset-v2.xlsx"
xl = pd.ExcelFile(file_path)

for sheet_name in xl.sheet_names:
    query_df = xl.parse(sheet_name, header=2)
    sheet_name = sheet_name.lower() 
    
    print(f"First 5 rows of sheet: {sheet_name}")
    print(query_df.head()) 
    print("\n")

First 5 rows of sheet: glucose in blood
  loinc_num                                   long_common_name  \
0    1988-5  C reactive protein [Mass/volume] in Serum or P...   
1    1959-6                Bicarbonate [Moles/volume] in Blood   
2   10331-7                                 Rh [Type] in Blood   
3   18998-5     Trimethoprim+Sulfamethoxazole [Susceptibility]   
4    1975-2   Bilirubin.total [Mass/volume] in Serum or Plasma   

                       component    system property  
0             C reactive protein  Ser/Plas     MCnc  
1                    Bicarbonate       Bld     SCnc  
2                             Rh       Bld     Type  
3  Trimethoprim+Sulfamethoxazole   Isolate     Susc  
4                      Bilirubin  Ser/Plas     MCnc  


First 5 rows of sheet: bilirubin in plasma
  loinc_num                                   long_common_name  \
0     934-0                          Blood product unit ID [#]   
1    1742-6  Alanine aminotransferase [Enzymatic activity/v...

# Data Cleaning: Renaming and Extracting Measurement Type

This script performs two data cleaning operations:

1. **Renaming Columns**  
2. **Extracting and Removing Measurement Types from Names**


In [9]:
for sheet_name in xl.sheet_names:
    query_df = xl.parse(sheet_name, header=2)
    sheet_name = sheet_name.lower() 
    query_df.rename(columns={"long_common_name": "name"}, inplace=True)

    query_df["measurement_type"] = query_df["name"].apply(lambda x: re.findall(r"\[(.*?)\]", x)[0] if "[" in x else "")
    query_df["name"] = query_df["name"].apply(lambda x: re.sub(r"\[.*?\]", "", x).strip() if isinstance(x, str) else x)

    print(f"First 5 rows of sheet: {sheet_name}")
    print(query_df.head()) 
    print("\n")

First 5 rows of sheet: glucose in blood
  loinc_num                                    name  \
0    1988-5  C reactive protein  in Serum or Plasma   
1    1959-6                   Bicarbonate  in Blood   
2   10331-7                            Rh  in Blood   
3   18998-5           Trimethoprim+Sulfamethoxazole   
4    1975-2     Bilirubin.total  in Serum or Plasma   

                       component    system property measurement_type  
0             C reactive protein  Ser/Plas     MCnc      Mass/volume  
1                    Bicarbonate       Bld     SCnc     Moles/volume  
2                             Rh       Bld     Type             Type  
3  Trimethoprim+Sulfamethoxazole   Isolate     Susc   Susceptibility  
4                      Bilirubin  Ser/Plas     MCnc      Mass/volume  


First 5 rows of sheet: bilirubin in plasma
  loinc_num                                          name  \
0     934-0                         Blood product unit ID   
1    1742-6  Alanine aminotransferas

# Abbreviation Mapping, Stop Words, and Lemmatization

This script performs **text preprocessing** by:
- Expanding **abbreviations** into full terms using the dictionary `abbreviation_mapping`.
- Removing **common stop words**.
- Applying **lemmatization** to reduce words to their base forms.

In [10]:
abbreviation_mapping = {
    'c': 'component',
    'mcnc': 'mass concentration',
    'bld': 'blood',
    'scnc': 'substance concentration',
    'susc': 'susceptibility',
    'acnc': 'amount concentration',
    'plas': 'plasma',
    'ccnc': 'cell concentration',
    'ncnc': 'number concentration',
    'XXX': 'unknown',
    '^bpu': 'body part or unit',
    'fld': 'field',
    'abo': 'abo blood group',
    'ser': 'serum',
    'mscnc': 'mass substance concentration'
}

nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joseantonioruizheredia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joseantonioruizheredia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Text Preprocessing in a DataFrame

This script performs text cleaning and abbreviation replacement on text columns in a DataFrame (`df`). It helps standardize text data for further analysis by removing noise and ensuring consistency.

## Functions

### 1. clean_text
Cleans a given text string by:
1. Converting it to **lowercase**.
2. Removing **punctuation** (replacing non-alphanumeric characters with spaces).
3. **Tokenizing** (splitting the text into words).
4. Removing **stop words** (common words that don't contribute much meaning).
5. Applying **lemmatization** (reducing words to their base form).

If the input is not a string, it returns an empty string.

### 2. replace_abbreviations
Replaces known abbreviations in a given text using the `abbreviation_mapping` dictionary.
- Splits the text into words.
- Replaces each word if it exists in the abbreviation dictionary.
- Returns the modified text.


In [11]:
def clean_text(text):
    if isinstance(text, str):
        text = text.lower() 
        text = re.sub(r'[^\w\s]', ' ', text)  
        words = text.split()  
        words = [word for word in words if word not in stop_words]  
        words = [lemmatizer.lemmatize(word) for word in words] 
        return " ".join(words)
    return ""


def replace_abbreviations(text):
    if isinstance(text, str):
        words = text.split()
        words = [abbreviation_mapping.get(word, word) for word in words]  
        return " ".join(words)
    return text


for col in query_df.select_dtypes(include=["object"]).columns:
    if col != "loinc_num":  
        query_df[col] = query_df[col].apply(clean_text)
        query_df[col] = query_df[col].apply(replace_abbreviations)

print(query_df.head(10))

  loinc_num                                               name  \
0   33870-7               bilirubin total unspecified specimen   
1   29265-6             calcium corrected albumin serum plasma   
2   14423-8                     bilirubin total synovial fluid   
3   23658-8                                         antibiotic   
4   19000-9                                         vancomycin   
5   14749-6                               glucose serum plasma   
6    1920-8            aspartate aminotransferase serum plasma   
7   18878-9                                          cefazolin   
8   20442-0  hepatitis b virus dna viral load serum probe s...   
9    1751-7                               albumin serum plasma   

                    component        system                 property  \
0                   bilirubin           xxx                    prthr   
1   calcium corrected albumin  serum plasma  substance concentration   
2                   bilirubin    synv field       mass co

# Column Weights and Embedding Model Initialization

This script defines **column weights** for a scoring system and initializes an **embedding model** for text similarity calculations.

## Column Weights

The `column_weights` dictionary assigns importance to different columns when calculating scores:

- **Higher weights** (e.g., `name`, `component`) indicate greater importance in the scoring process.
- `loinc_num` has a weight of **0** because it is likely an identifier and does not contribute to similarity calculations.

## Embedding Model Initialization

The script attempts to load an embedding model for text similarity using **SentenceTransformer**. 


In [12]:
column_weights = {
    'name': 1.5,
    'component': 6.0,
    'long_common_name': 1.0,
    'system': 3.0,
    'property': 1.0,
    'measurement_type': 1.0,
    'loinc_num': 0
}

global embedding_model
if 'embedding_model' not in globals():
    try:
        embedding_model = SentenceTransformer('pritamdeka/BioBERT-MNLI')
    except:
        try:
            embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        except:
            embedding_model = None

# Relevance Score Calculation
This script calculates a **relevance score** for each row in a dataset by comparing a query to the dataset's text fields using both **traditional keyword matching** and **semantic similarity via embeddings**.


### 1. calculate_score
This function calculates the relevance score for a given row by:
1. Splitting the query into words and storing them in a set.
2. Initializing an empty dictionary to track matched words.
3. Computing the **traditional** relevance score.
4. Computing the **embedding-based** relevance score.
5. Combining both scores and optionally printing debug information.


### 2. get_query_embedding
Encodes the query into an **embedding vector** using a pre-trained embedding model.
- If the embedding model is available, it encodes the query.
- If an error occurs, it prints an error message and returns `None`.

### 3. calculate_traditional_score
This function calculates a **keyword matching score** based on:
- The presence of query words in the main columns `component` and `system`
- A predefined weight assigned to each column.

### 4. calculate_embedding_score
This function computes the **semantic similarity score** between the query embedding and the row's text fields:
- Encodes the text field into an embedding.
- Uses **cosine similarity** to measure similarity between the query and field.
- Converts the similarity score (ranging from -1 to 1) into a normalized range.
- Applies column weights to adjust the score.
- Adds the result to the final score.



In [ ]:
def calculate_score(query, query_df, row, debug=False):
    query_embedding = get_query_embedding(query)
    debug_info = {"query": query, "embedding_score": [], "traditional_score": []}
    
    traditional_score = calculate_traditional_score(query, row, debug_info)
    embedding_score = calculate_embedding_score(query_embedding, query_df, row, debug_info)
    score = traditional_score + embedding_score
    
    debug_info["final_score"] = score
    
    if debug:
        print(debug_info)
   
    return score


def get_query_embedding(query):
    if embedding_model:
        try:
            return embedding_model.encode(query.lower())
        except Exception as e:
            print(f"Embedding encoding error: {e}")
    return None

def calculate_traditional_score(query, row, debug_info):
    score = 0
    
    query_component = query_mapping[query]["component"].lower()
    query_system = query_mapping[query]["system"].lower()

    component = row.get("component", "").lower() 
    system = row.get("system", "").lower()  
    
    if query_component == component:
        score += column_weights.get("component", 1.0)  * column_weights.get("component", 1.0) 
    elif query_component in component:
        score += (column_weights.get("component", 1.0) * 0.5) * column_weights.get("component", 1.0)   
    
    if query_system == system:
        score += column_weights.get("system", 1.0)  * column_weights.get("system", 1.0)  
    elif query_system in system:
        score += (column_weights.get("system", 1.0) * 0.5) * column_weights.get("system", 1.0)   

    debug_info["traditional_score"].append({"score": score})
        
    return score

def calculate_embedding_score(query_embedding, query_df, row, debug_info):
    score = 0
    if embedding_model and query_embedding is not None:
        for col in query_df.select_dtypes(include=["object"]).columns:
            if col in row and pd.notna(row[col]):
                cell_text = str(row[col]).lower()
                weight = column_weights.get(col, 1.0)
                try:
                    cell_embedding = embedding_model.encode(cell_text)
                    similarity = cosine_similarity([query_embedding], [cell_embedding])[0][0]
                    embedding_score = ((similarity + 1) / 2) * 5 * weight
                    score += embedding_score
                except Exception as e:
                    print(f"Embedding similarity error: {e}")
        debug_info["embedding_score"].append({"score": score})
    return score


# Preprocessing of the file
This script processes an Excel file, calculates relevance scores for each row based on a query, normalizes the scores, and saves the results to a CSV file. The relevance scores are computed using both traditional keyword matching and embeddings.

### 1. Data Preprocessing
   - Loads the Excel file and iterates over each sheet.
   - Renames columns for consistency and extracts relevant information (e.g., measurement type) from the `name` column.
   - Cleans and strips spaces from column names.

### 2. Score Calculation
   - For each row, a relevance score is computed using `calculate_score`.
   - Debug information is printed for the first 5 rows.

### 3.Normalization
   - The minimum and maximum scores are calculated, and a new column for normalized scores is added using the formula `(score - min_score) / (max_score - min_score)`.

### 4. Results
   - The results (Query, LOINC Code, Name, Score, Normalized Score) are saved to a CSV file: `dataset_with_scores.csv`.



In [14]:
min_score = float("inf")
max_score = float("-inf")


def preprocess(excel_file):
    results = []
    xl = pd.ExcelFile(excel_file)

    for sheet_name in xl.sheet_names:
        query_df = xl.parse(sheet_name, header=2)
        sheet_name = sheet_name.lower()
         
        query_df.rename(columns={"long_common_name": "name"}, inplace=True)
        query_df["measurement_type"] = query_df["name"].apply(lambda x: re.findall(r"\[(.*?)\]", x)[0] if "[" in x else "")
        query_df["name"] = query_df["name"].apply(lambda x: re.sub(r"\[.*?\]", "", x).strip() if isinstance(x, str) else x)

        for col in query_df.select_dtypes(include=["object"]).columns:
            if col != "loinc_num":  
                query_df[col] = query_df[col].apply(clean_text)
                query_df[col] = query_df[col].apply(replace_abbreviations)
            
        query_df.columns = query_df.columns.str.strip()  
        
        if len(query_df.columns) < 4:
            print(f"Skipping sheet '{sheet_name}': Missing required columns")
            continue
        
        for _, row in query_df.iterrows():
            score = calculate_score(sheet_name, query_df, row, debug=True if row.name < 5 else False)
            results.append([sheet_name, row.iloc[0], row.iloc[1], row.iloc[2], row.iloc[3], row.iloc[4], row.iloc[5], score])

    results_df = pd.DataFrame(results, columns=["Query", "LOINC Code", "Name", "Component", "System", "Property", "Measurement", "Score"])

    min_score = results_df["Score"].min()
    max_score = results_df["Score"].max()

    results_df["Normalized_Score"] = results_df["Score"].apply(lambda score: (score - min_score) / (max_score - min_score) if max_score != min_score else 1.0)
    results_df.drop(columns=["Score"], inplace=True)

    results_df.to_csv("dataset_with_scores.csv", index=False)
    print("CSV with relevance scores has been saved as 'dataset_with_scores.csv'.")



results_df = preprocess(file_path)

{'query': 'glucose in blood', 'embedding_score': [{'score': np.float32(35.774864)}], 'traditional_score': [{'score': 0}], 'final_score': np.float32(35.774864)}
{'query': 'glucose in blood', 'embedding_score': [{'score': np.float32(38.686966)}], 'traditional_score': [{'score': 9.0}], 'final_score': np.float32(47.686966)}
{'query': 'glucose in blood', 'embedding_score': [{'score': np.float32(36.613216)}], 'traditional_score': [{'score': 9.0}], 'final_score': np.float32(45.613216)}
{'query': 'glucose in blood', 'embedding_score': [{'score': np.float32(32.27971)}], 'traditional_score': [{'score': 0}], 'final_score': np.float32(32.27971)}
{'query': 'glucose in blood', 'embedding_score': [{'score': np.float32(37.938316)}], 'traditional_score': [{'score': 0}], 'final_score': np.float32(37.938316)}
{'query': 'bilirubin in plasma', 'embedding_score': [{'score': np.float32(37.021507)}], 'traditional_score': [{'score': 0}], 'final_score': np.float32(37.021507)}
{'query': 'bilirubin in plasma', 'e